In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

train = pd.read_csv('../input/train.tsv',sep='\t')

In [ ]:
test = pd.read_csv('../input/test.tsv',sep='\t')

In [ ]:
# pre processing

## train test combine

target = train.price
train_features = train.drop('price',axis=1)
train_features = train_features.rename(columns={'train_id':'id'})
test = test.rename(columns={'test_id':'id'})
train_features['is_train'] = 1
test['is_train'] = 0
train_test = pd.concat([train_features,test],axis=0)

train_test.head()

In [ ]:
def get_dummies_and_drop_dummied_columns(df,columnsName):
    dummy_df = pd.get_dummies(df[columnsName],prefix=columnsName,drop_first=True)
    df =  pd.concat([df,dummy_df],axis=1).drop(columnsName,axis=1)
    return df

def pre_processing(df):
    #欠損値補完
    #df.brand_name = df.brand_name.fillna('no_brand')
    df.category_name = df.category_name.fillna('no_cate1/no_cate2/no_cate3')
    #df.item_description = df.item_description.fillna('n')

    #Category_nameをわける
    df['cate1'], df['cate2'], df['cate3'] = zip(*df.category_name.apply(lambda x : x.split('/')))
    df.brand_name = df.brand_name.astype('category').cat.codes
    df.cate1 = df.cate1.astype('category').cat.codes
    df.cate2 = df.cate2.astype('category').cat.codes
    df.cate3 = df.cate3.astype('category').cat.codes
    
    #name, item_descriptionの長さを追加する
    df['name_length'] = df.name.apply(lambda x : len(x))
    df['desc_length'] = df.name.apply(lambda x : len(x))

    # ダミー変数化
    df = get_dummies_and_drop_dummied_columns(df,'item_condition_id')
    #df = get_dummies_and_drop_dummied_columns(df,'cate1')
    #df = get_dummies_and_drop_dummied_columns(df,'cate2')
    #df = get_dummies_and_drop_dummied_columns(df,'cate3')
    #df = get_dummies_and_drop_dummied_columns(df,'brand_name')
    
    #不要なカラムの削除
    df.drop(['name','category_name','item_description'],axis=1,inplace=True)
    return df

In [ ]:
train_test_preprocessed = pre_processing(train_test)

In [ ]:
## separate train and test

train = train_test_preprocessed[train_test_preprocessed.is_train == 1].reset_index(drop=True).drop('is_train',axis=1)

test = train_test_preprocessed[train_test_preprocessed.is_train == 0].reset_index(drop=True).drop('is_train',axis=1)

# RandomForestRegression
features = train.drop('id',axis=1)
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=200,n_jobs=-1)
model.fit(features,target)

In [ ]:
test = test.drop('id',axis=1)
predict_test= model.predict(test)
test['test_id'] = range(len(test))
submission = pd.concat([test[['test_id']],pd.DataFrame(predict_test)],axis=1)
submission.columns = ['test_id', 'price']
submission.to_csv('submission.csv',index=False)

In [ ]:
submission.head()